This script computes the average NDVI for each green area in the municipality of Bologna

### 0. Libraries

In [ ]:
# Libraries
import geopandas as gpd
import rasterio
import numpy as np
from rasterio.features import geometry_mask
from rasterio.warp import calculate_default_transform, reproject, Resampling


### 1. NDVI

In [2]:
# Load the shapefile
green_areas = gpd.read_file(r"D:\Climate_Shelter_Index\ar_vrd_s_df_pt_b.shp")

In [ ]:
# Open the GeoTIFF
with rasterio.open("D:/Climate_Shelter_Index/NDVI/ndvi_bologna.tif") as src:
    # Define the target CRS (EPSG:4326)
    target_crs = 'EPSG:4326'

    # Calculate the transformation parameters for reprojection
    transform, width, height = calculate_default_transform(src.crs, target_crs, src.width, src.height, *src.bounds)

    # Create an empty array for the reprojected data
    reprojected_ndvi = np.empty((height, width), dtype=src.dtypes[0])

    # Reproject the data to EPSG:4326
    reproject(
        source=rasterio.band(src, 1),
        destination=reprojected_ndvi,
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=transform,
        dst_crs=target_crs,
        resampling=Resampling.bilinear
    )

    # Iterate through green areas
    for index, row in green_areas.iterrows():
        # Get the geometry of the green area
        geometry = row['geometry']
        
        # Create a mask for the reprojected NDVI values based on the green area's geometry
        mask = geometry_mask([geometry], out_shape=reprojected_ndvi.shape, transform=transform, invert=True)
        
        # Extract NDVI values within the green area using the mask
        ndvi_values_within_area = np.ma.masked_array(reprojected_ndvi, mask=~mask)
        
        # Calculate the mean NDVI for the green area while ignoring masked (NaN) values
        mean_ndvi = np.mean(ndvi_values_within_area)
        
        # Update the 'NDVI' column with the computed mean NDVI value
        green_areas.at[index, 'NDVI'] = mean_ndvi

In [8]:
green_areas.head()

,quart,nome,ubicazione,classe_uni,classe_gia,classe_pen,classe_suo,area_prato,area_ug,data_agg,...,zona_pross,area_Ha,d_fountain,id_df,picnic_tab,id_pt,benches,id_bench,geometry,NDVI
0,Santo Stefano,PARCO DEI CALANCHI DI SABBIUNO,Via di Sabbiuno,PARCO ESTENSIVO,SOMMITALE,0 -20%,ARGILLOSO COMPATTO,9509.007,12461.168,2009-01-29,...,OSSERVANZA - PADERNO,0.950901,no,N/A,no,N/A,no,N/A,"POLYGON ((11.31509 44.42248, 11.31511 44.42256...",0.542155
1,Santo Stefano,GIARDINO MONUMENTO CADUTI DI SABBIUNO,Via di Sabbiuno,PARCO,SOMMITALE,0 -20%,ARGILLOSO COMPATTO,5514.205,6730.89,2009-01-29,...,OSSERVANZA - PADERNO,0.551420,no,N/A,no,N/A,no,N/A,"POLYGON ((11.31288 44.42104, 11.31289 44.42109...",0.459359
2,Santo Stefano,PARCO DEI CALANCHI DI SABBIUNO,Via di Sabbiuno,PARCO ESTENSIVO,SOMMITALE,0 -20%,ARGILLOSO COMPATTO,16811.194,24817.745,2007-03-13,...,OSSERVANZA - PADERNO,1.681119,no,N/A,no,N/A,no,N/A,"POLYGON ((11.31463 44.42146, 11.31496 44.42149...",0.624308
3,Santo Stefano,PARCO PADERNO,Via Paderno,PARCO ESTENSIVO,PIANO,20 - 40%,MEDIO IMPASTO,87.272,6250.574,2007-09-20,...,OSSERVANZA - PADERNO,0.008727,no,N/A,no,N/A,no,N/A,"POLYGON ((11.32362 44.45097, 11.32372 44.45110...",0.877684
4,Santo Stefano,PARCO PADERNO,Via Paderno,PARCO ESTENSIVO,PIANO,20 - 40%,MEDIO IMPASTO,-4.817,21360.397,2007-09-20,...,OSSERVANZA - PADERNO,-0.000482,no,N/A,no,N/A,no,N/A,"POLYGON ((11.32160 44.45340, 11.32164 44.45341...",0.879811


In [9]:
# Save the updated shapefile with the 'NDVI' column included
green_areas.to_file('D:/Climate_Shelter_Index/green_areas__ndvi.shp')